In [1]:
import os
import urllib.request
from rdflib import Graph
from rdflib.namespace import SKOS

In [2]:
onto_url = "https://raw.githubusercontent.com/emmo-repo/domain-electrochemistry/master/electrochemistry.ttl"
data_path = "./data/"
example_filename = "example.ttl"
if example_filename not in os.listdir(data_path):
    urllib.request.urlretrieve(onto_url, data_path+example_filename)


In [3]:
# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
g.parse(data_path+example_filename,format="ttl")
len(g)

2254

In [4]:
names = g.subject_objects(predicate=SKOS.prefLabel)
for subject, object in names:
    print(subject, " : ", object)

http://emmo.info/electrochemistry#electrochemistry_3bd08946_4e81_455d_9fca_dc7a5ead9315  :  hasElectrolyte
http://emmo.info/electrochemistry#electrochemistry_578c41e9_ee01_4840_9c8c_04ab6e4e6241  :  hasElectrode
http://emmo.info/electrochemistry#electrochemistry_5d299271_3f68_494f_ab96_3db9acdd3138  :  hasNegativeElectrode
http://emmo.info/electrochemistry#electrochemistry_5eb197ea_0c3e_4ea2_8392_81152ee91515  :  hasReferenceElectrode
http://emmo.info/electrochemistry#electrochemistry_6bf73b4c_bc73_4db1_bf51_12628ce9ea3b  :  hasElectrodeConstituent
http://emmo.info/electrochemistry#electrochemistry_860aa941_5ff9_4452_8a16_7856fad07bee  :  hasActiveMaterial
http://emmo.info/electrochemistry#electrochemistry_8e9cf965_9f92_46e8_b678_b50410ce3616  :  hasPositiveElectrode
http://emmo.info/electrochemistry#electrochemistry_008201ec_1e44_47a9_bc50_644ed07489ea  :  Electrogravimetry
http://emmo.info/electrochemistry#electrochemistry_14d0358f_f65d_44f9_8b85_40b38f17c4ed  :  Electrogravimetry
ht

## Haystack

In [9]:
from haystack.utils import fetch_archive_from_http
from pathlib import Path
from haystack.document_stores import InMemoryKnowledgeGraph
from haystack.nodes import Text2SparqlRetriever



In [6]:
model_dir = "./model/"
if not os.listdir(model_dir):
    s3_url = "https://fandom-qa.s3-eu-west-1.amazonaws.com/saved_models/hp_v3.4.zip"
    fetch_archive_from_http(url=s3_url, output_dir=model_dir)

True

In [7]:
not os.listdir(model_dir)

False

In [13]:
# Initialize a in memory knowledge graph and use "tutorial_10_index" as the name of the index
kg = InMemoryKnowledgeGraph(index="tutorial_10_index")

# Delete the index as it might have been already created in previous runs
kg.delete_index()

# Create the index
kg.create_index()

# Import triples of subject, predicate, and object statements from a ttl file
kg.import_from_ttl_file(index="tutorial_10_index", path=os.path.join(data_path, example_filename))
print(f"The last triple stored in the knowledge graph is: {kg.get_all_triples()[-1]}")
print(f"There are {len(kg.get_all_triples())} triples stored in the knowledge graph.")

The last triple stored in the knowledge graph is: {'s': {'type': 'uri', 'value': 'http://emmo.info/electrochemistry#electrochemistry_5641910f_6e69_4ce4_be84_4b1bf14b8916'}, 'p': {'type': 'uri', 'value': 'http://emmo.info/emmo#EMMO_50c298c2_55a2_4068_b3ac_4e948c33181f'}, 'o': {'type': 'uri', 'value': 'https://www.electropedia.org/iev/iev.nsf/display?openform&ievref=114-04-01'}}
There are 2254 triples stored in the knowledge graph.


In [14]:
# Load a pre-trained model that translates text queries to SPARQL queries
kgqa_retriever = Text2SparqlRetriever(knowledge_graph=kg, model_name_or_path=Path(model_dir) / "hp_v3.4")

In [17]:
query = "What is Electrode?"
print(f'Translating the text query "{query}" to a SPARQL query and executing it on the knowledge graph...')
result = kgqa_retriever.retrieve(query=query)
print(result)


Translating the text query "What is Electrode?" to a SPARQL query and executing it on the knowledge graph...
[{'answer': '', 'prediction_meta': {'model': 'Text2SparqlRetriever', 'sparql_query': ''}}]


### Conclusion
Haystack does not work out of the box for our ontologies. We need to explore a more human-readable representation of the IRIs (using prefixes and aliases?).